# Airline On-time Performance Dashboard with Plotly-Dash

## Dependencies

In [ ]:
%pip install -q pandas requests lxml tabulate pyarrow

## Imports

In [ ]:
import pandas as pd
import requests
import tarfile
from os import path

## Airline Reporting Carrier On-Time Performance Dataset

The Reporting Carrier On-Time Performance Dataset contains US domestic flights between 1987 and 2020.

The Reporting Carrier On-Time Performance Dataset contains information on approximately 200 million domestic US flights reported to the United States Bureau of Transportation Statistics. The dataset contains basic information about each flight (such as date, time, departure airport, arrival airport) and, if applicable, the amount of time the flight was delayed and information about the reason for the delay. This dataset can be used to predict the likelihood of a flight arriving on time.

### Columns

In [ ]:
url_data_summary = (
    "https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/data-preview/index.html"
)
columns = pd.read_html(url_data_summary)[2]
columns.to_markdown("columns.md")

### Loading data

In [ ]:
# # Downloading the dataset

# fname = 'airline_2m.tar.gz'
# # fname = 'airline.tar.gz'
# url = 'https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/' + fname
# r = requests.get(url)
# open(fname , 'wb').write(r.content)

In [ ]:
# # Extracting the dataset
# tar = tarfile.open(fname)
# tar.extractall()
# tar.close()

# # Verifying the file was extracted properly
data_path = "airline_2m.csv"
# path.exists(data_path)

In [ ]:
df = pd.read_csv(
    data_path,
    nrows=10_000,
    encoding="ISO-8859-1",
)

# Preview the first 5 lines of the loaded data
df.head()

In [ ]:
df.shape

### Column types

In [ ]:
pd.options.display.max_columns = 109
df.dtypes.to_frame().T

### Comparing size of the dataset with PyArrow

In [ ]:
def clean_data(df: pd.DataFrame):
    # Change column type to int16 for column: 'DestAirportSeqID'
    df = df.convert_dtypes()
    return df


df_clean = clean_data(df.copy())
df_clean.head()

In [ ]:
df_py = pd.read_csv(
    data_path,
    nrows=10_000,
    encoding="ISO-8859-1",
    dtype_backend="pyarrow",
)

# categories columns
cols_cat = ['Div1Airport', 'DistanceGroup', 'Diverted', 'CancellationCode', 'Cancelled', 'ArrivalDelayGroups', 'ArrDel15', 'DepartureDelayGroups', 'DepDel15', 'DestStateName', 'DestState', 'DestCityName', 'Dest', 'OriginStateName', 'OriginState', 'OriginCityName', 'Origin', 'IATA_CODE_Reporting_Airline','Reporting_Airline', 'DayOfWeek', 'Month', 'Quarter']
df_py[cols_cat] = df_py[cols_cat].astype('category')

# unsigned columns
cols_unsigned = ['Year', 'DayofMonth', 'DOT_ID_Reporting_Airline', 'Flight_Number_Reporting_Airline', 'OriginAirportID', 'OriginAirportSeqID', 'OriginCityMarketID', 'OriginStateFips', 'OriginWac', 'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'DestStateFips', 'DestWac', 'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn', 'CRSArrTime', 'ArrTime', 'ArrDelayMinutes', 'CRSElapsedTime', 'ActualElapsedTime', 'AirTime', 'Flights', 'Distance', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'FirstDepTime', 'TotalAddGTime', 'LongestAddGTime', 'DivAirportLandings', 'DivActualElapsedTime', 'DivArrDelay', 'DivDistance', 'Div1AirportID', 'Div1AirportSeqID', 'Div1WheelsOn', 'Div1TotalGTime', 'Div1LongestGTime', 'Div1WheelsOff']
for col in cols_unsigned:
    df_py[col] = pd.to_numeric(df_py[col], downcast='unsigned')

# signed columns
cols_signed = ['ArrDelay', 'DepDelay']
for col in cols_signed:
    df_py[col] = pd.to_numeric(df_py[col], downcast='signed')


df_py.info()

In [ ]:
df_mem = pd.DataFrame(
    {
        "df": df.dtypes,
        "df_clean": df_clean.dtypes,
        "df_py": df_py.dtypes,
        "df_mem": df.memory_usage(),
        "df_clean_mem": df_clean.memory_usage(),
        "df_py_mem": df_py.memory_usage(),
    }
).T
df_mem["Total"] = df_mem.T.loc[:, ["df_mem", "df_clean_mem", "df_py_mem"]].sum()

df_mem